This is another kernel from the 2nd YouTube Competition- could maybe give us ideas of how to set up a model for the 3rd.

In [5]:
from multiprocessing import Pool, cpu_count
from IPython.display import YouTubeVideo #YouTubeVideo('-0OWhcdBt0k', 7)
from sklearn import ensemble, metrics, preprocessing
from sklearn.ensemble import RandomForestClassifier
import tensorflow as tf
import pandas as pd
import numpy as np
import glob

In [17]:
[x for x in pd.read_csv('sample/vocabulary.csv').loc[:,['Index','Name']]]

['Index', 'Name']

In [18]:
sub = pd.read_csv('sample/sample_submission.csv')
# lbl = {k:{'label':v, 'count':0} for k,v in pd.read_csv('sample/vocabulary.csv')[['Index','Name']]}
train_videos = glob.glob('yt8m/v2/video/train*')
test_videos = glob.glob('yt8m/v2/video/test*')
val_videos = glob.glob('yt8m/v2/video/val*')
frames = glob.glob('yt8m/v2/frame/*')
print(len(train_videos), len(test_videos), len(val_videos),len(frames))

41 40 39 123


In [19]:
models = []
train = []
loops_ = 0
for tf_vids in train_videos:
    #f = tf_vids.split('/')[-1].split('.')[0]
    if loops_ % 10 == 0:
        print(loops_)
    for tf_zip in tf.python_io.tf_record_iterator(tf_vids):
        video = tf.train.Example.FromString(tf_zip)
        vid_id = video.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8')
        mean_rgb = video.features.feature['mean_rgb'].float_list.value
        mean_audio = video.features.feature['mean_audio'].float_list.value
        l = ','.join(map(str, video.features.feature['labels'].int64_list.value))
        train.append([vid_id, l]+ list(mean_rgb) + list(mean_audio))
    if loops_ > 2:
        break
    loops_ += 1

col = ['VideoId','label'] + ['mean_rgb'+str(i) for i in range(1024)] + ['mean_audio'+str(i) for i in range(128)]
train = pd.DataFrame(train, columns=col)
col = [c for c in col if c not in ['VideoId','label']]
y = train['label'].str.get_dummies(sep=',')
model =  RandomForestClassifier(min_samples_split = 40, max_leaf_nodes = 15, n_estimators = 40, max_depth = 5,min_samples_leaf = 3)
sc = preprocessing.StandardScaler()
model.fit(sc.fit_transform(train[col]), y)
models.append([model, y.columns])

0
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
